In [61]:
from IPython.display import HTML
HTML('''
    <style> body {font-family: "Roboto Condensed Light", "Roboto Condensed";} h2 {padding: 10px 12px; background-color: #E64626; position: static; color: #ffffff; font-size: 40px;} .text_cell_render p { font-size: 15px; } .text_cell_render h1 { font-size: 30px; } h1 {padding: 10px 12px; background-color: #E64626; color: #ffffff; font-size: 40px;} .text_cell_render h3 { padding: 10px 12px; background-color: #0148A4; position: static; color: #ffffff; font-size: 20px;} h4:before{ 
    content: "@"; font-family:"Wingdings"; font-style:regular; margin-right: 4px;} .text_cell_render h4 {padding: 8px; font-family: "Roboto Condensed Light"; position: static; font-style: italic; background-color: #FFB800; color: #ffffff; font-size: 18px; text-align: center; border-radius: 5px;}input[type=submit] {background-color: #E64626; border: solid; border-color: #734036; color: white; padding: 8px 16px; text-decoration: none; margin: 4px 2px; cursor: pointer; border-radius: 20px;}</style>
    <script> code_show=true; function code_toggle() {if (code_show){$('div.input').hide();} else {$('div.input').show();} code_show = !code_show} $( document ).ready(code_toggle);</script>
    <form action="javascript:code_toggle()"><input type="submit" value="Hide/show all code."></form>
''')

# Sydney Liveability Analysis

## Group - 5  F10Adv-01

In [62]:
import pandas as pd
import geopandas as gpd
import json
from shapely.geometry import Point, Polygon, MultiPolygon
from geoalchemy2 import Geometry, WKTElement
import matplotlib.pyplot as plt


import geoplot as gplt
import geopandas as gpd
import geoplot.crs as gcrs
import imageio
import pandas as pd
import pathlib
import matplotlib.pyplot as plt
import mapclassify as mc
import numpy as np

In [63]:
sa2_data = gpd.read_file("SA2_2016_AUST/SA2_2016_AUST.shp")

In [64]:
break_and_enter = gpd.read_file("break_and_enter/BreakEnterDwelling_JanToDec2021.shp")

In [65]:
school_catchments_future = gpd.read_file("school_catchments/catchments_future.shp")

In [66]:
school_catchments_primary = gpd.read_file("school_catchments/catchments_primary.shp")

In [67]:
school_catchments_secondary = gpd.read_file("school_catchments/catchments_secondary.shp")

In [68]:
neighbourhoods = pd.read_csv('Neighbourhoods.csv')


In [69]:
BusinessStats = pd.read_csv('BusinessStats.csv')

In [70]:
Playgrounds_json = json.load(open('Playgrounds.geojson'))

Playgrounds_DataFrame = pd.DataFrame(Playgrounds_json["features"])
Playgrounds_pandas_data=[]
for index, row in Playgrounds_DataFrame.iterrows():
    append_row = [row["geometry"]["coordinates"][0],row["geometry"]["coordinates"][1],row["properties"]["Name"]]
    Playgrounds_pandas_data.append(append_row)
    
Playgrounds = pd.DataFrame(Playgrounds_pandas_data, columns = ['X', 'Y','Name'])
Playgrounds['geom'] = gpd.points_from_xy(Playgrounds.X, Playgrounds.Y)  # creating the geometry column
Playgrounds = Playgrounds.drop(columns=['X', 'Y'])  # removing the old latitude/longitude fields

FileNotFoundError: [Errno 2] No such file or directory: 'Playgrounds.geojson'

In [ ]:
Lights_json = json.load(open('Lights.geojson'))
Lights_DataFrame = pd.DataFrame(Lights_json["features"])
Lights_pandas_data=[]
for index, row in Lights_DataFrame.iterrows():
    append_row = [row["geometry"]["coordinates"][0],row["geometry"]["coordinates"][1],row["properties"]["Location"]]
    Lights_pandas_data.append(append_row)
    
Lights = pd.DataFrame(Lights_pandas_data, columns = ['X', 'Y','Location'])
Lights['geom'] = gpd.points_from_xy(Lights.X, Lights.Y)  # creating the geometry column
Lights = Lights.drop(columns=['X', 'Y'])  # removing the old latitude/longitude fields

In [ ]:
from sqlalchemy import create_engine
import psycopg2
import psycopg2.extras
import json

credentials = "Credentials.json"

def pgconnect(credential_filepath, db_schema="public"):
    with open(credential_filepath) as f:
        db_conn_dict = json.load(f)
        host       = db_conn_dict['host']
        db_user    = db_conn_dict['user']
        db_pw      = db_conn_dict['password']
        default_db = db_conn_dict['user']
        try:
            db = create_engine('postgresql+psycopg2://'+db_user+':'+db_pw+'@'+host+'/'+default_db, echo=False)
            conn = db.connect()
            print('Connected successfully.')
        except Exception as e:
            print("Unable to connect to the database.")
            print(e)
            db, conn = None, None
        return db,conn

def query(conn, sqlcmd, args=None, df=True):
    result = pd.DataFrame() if df else None
    try:
        if df:
            result = pd.read_sql_query(sqlcmd, conn, params=args)
        else:
            result = conn.execute(sqlcmd, args).fetchall()
            result = result[0] if len(result) == 1 else result
    except Exception as e:
        print("Error encountered: ", e, sep='\n')
    return result

In [ ]:
db, conn = pgconnect(credentials)

Connected successfully.


In [ ]:
query(conn, "select PostGIS_Version()")

,postgis_version
0,3.0 USE_GEOS=1 USE_PROJ=1 USE_STATS=1


In [ ]:
srid = 4326
Playgrounds['geom'] = Playgrounds['geom'].apply(lambda x: WKTElement(x.wkt, srid=srid))
Lights['geom'] = Lights['geom'].apply(lambda x: WKTElement(x.wkt, srid=srid))

In [ ]:
def create_wkt_element(geom, srid):
    if geom!=None :
        if geom.geom_type == 'Polygon':
            geom = MultiPolygon([geom])
        return WKTElement(geom.wkt, srid)

sa2_data_og = sa2_data.copy()  # creating a copy of the original for later
sa2_data['geom'] = sa2_data['geometry'].apply(lambda x: create_wkt_element(geom=x,srid=srid))  # applying the function
sa2_data = sa2_data.drop(columns="geometry")  # deleting the old copy

break_and_enter_og = break_and_enter.copy()  # creating a copy of the original for later
break_and_enter['geom'] = break_and_enter['geometry'].apply(lambda x: create_wkt_element(geom=x,srid=srid))  # applying the function
break_and_enter = break_and_enter.drop(columns="geometry")  # deleting the old copy

school_catchments_future_og = school_catchments_future.copy()  # creating a copy of the original for later
school_catchments_future['geom'] = school_catchments_future['geometry'].apply(lambda x: create_wkt_element(geom=x,srid=srid))  # applying the function
school_catchments_future = school_catchments_future.drop(columns="geometry")  # deleting the old copy

school_catchments_primary_og = school_catchments_primary.copy()  # creating a copy of the original for later
school_catchments_primary['geom'] = school_catchments_primary['geometry'].apply(lambda x: create_wkt_element(geom=x,srid=srid))  # applying the function
school_catchments_primary = school_catchments_primary.drop(columns="geometry")  # deleting the old copy

school_catchments_secondary_og = school_catchments_secondary.copy()  # creating a copy of the original for later
school_catchments_secondary['geom'] = school_catchments_secondary['geometry'].apply(lambda x: create_wkt_element(geom=x,srid=srid))  # applying the function
school_catchments_secondary = school_catchments_secondary.drop(columns="geometry")  # deleting the old copy

In [ ]:
print(type(sa2_data))
print(type(BusinessStats))
#BusinessStats

<class 'geopandas.geodataframe.GeoDataFrame'>
<class 'pandas.core.frame.DataFrame'>


In [ ]:
# conn.execute("""
# CREATE SCHEMA IF NOT EXISTS Sydney_Liveability_Analysis;
# SET search_path TO Sydney_Liveability_Analysis;
# """)

# sa2_data
conn.execute("""
DROP TABLE IF EXISTS sa2_data;
CREATE TABLE sa2_data (
    "SA2_MAIN16" INTEGER, 
    "SA2_5DIG16" INTEGER, 
    "SA2_NAME16" VARCHAR(80),
    "SA3_CODE16" INTEGER,
    "SA3_NAME16" VARCHAR(80),
    "SA4_CODE16" INTEGER,
    "SA4_NAME16" VARCHAR(80),
    "GCC_CODE16" VARCHAR(80),
    "GCC_NAME16" VARCHAR(80),
    "STE_CODE16" INTEGER,
    "STE_NAME16" VARCHAR(80),	
    "AREASQKM16" NUMERIC,
    "geom" GEOMETRY(MULTIPOLYGON,4326)
);"""
)

# neighbourhood
conn.execute("""
DROP TABLE IF EXISTS neighbourhood;
CREATE TABLE neighbourhood (
    "Unnamed: 0" INTEGER,
    "area_id" INTEGER,
    "area_name" VARCHAR(80),
    "land_area" NUMERIC,
    "population" VARCHAR(80),
    "number_of_dwellings" VARCHAR(80),
    "number_of_businesses" NUMERIC,
    "median_annual_household_income" NUMERIC,
    "avg_monthly_rent" INTEGER,
    "0-4" INTEGER,
    "5-9" INTEGER,
    "10-14" INTEGER,
    "15-19" INTEGER 
);"""
)

# break and enter
conn.execute("""
DROP TABLE IF EXISTS break_and_enter;
CREATE TABLE break_and_enter (
    "OBJECTID" INTEGER, 
    "Contour" NUMERIC, 
    "Density" VARCHAR(80), 
    "ORIG_FID" INTEGER, 
    "Shape_Leng" NUMERIC,
    "Shape_Area" NUMERIC,
    "geom" GEOMETRY(MULTIPOLYGON,4326)
);"""
)

# school_catchments_future
conn.execute("""
DROP TABLE IF EXISTS school_catchments_future;
CREATE TABLE school_catchments_future (
    "USE_ID" INTEGER, 
    "CATCH_TYPE" VARCHAR(80), 
    "USE_DESC" VARCHAR(80), 
    "ADD_DATE" INTEGER, 
    "KINDERGART" INTEGER,
    "YEAR1" INTEGER,
    "YEAR2" INTEGER,
    "YEAR3" INTEGER,
    "YEAR4" INTEGER,
    "YEAR5" INTEGER,
    "YEAR6" INTEGER,
    "YEAR7" INTEGER,
    "YEAR8" INTEGER,
    "YEAR9" INTEGER,
    "YEAR10" INTEGER,
    "YEAR11" INTEGER,
    "YEAR12" INTEGER,
    "geom" GEOMETRY(MULTIPOLYGON,4326)
    
);"""
)

# school_catchments_primary
conn.execute("""
DROP TABLE IF EXISTS school_catchments_primary;
CREATE TABLE school_catchments_primary (
    "USE_ID" INTEGER, 
    "CATCH_TYPE" VARCHAR(80), 
    "USE_DESC" VARCHAR(80), 
    "ADD_DATE" INTEGER, 
    "KINDERGART" VARCHAR(1),
    "YEAR1" VARCHAR(1),
    "YEAR2" VARCHAR(1),
    "YEAR3" VARCHAR(1),
    "YEAR4" VARCHAR(1),
    "YEAR5" VARCHAR(1),
    "YEAR6" VARCHAR(1),
    "YEAR7" VARCHAR(1),
    "YEAR8" VARCHAR(1),
    "YEAR9" VARCHAR(1),
    "YEAR10" VARCHAR(1),
    "YEAR11" VARCHAR(1),
    "YEAR12" VARCHAR(1),
    "PRIORITY" VARCHAR(80),
    "geom" GEOMETRY(MULTIPOLYGON,4326)
    
);"""
)

# school_catchments_secondary
conn.execute("""
DROP TABLE IF EXISTS school_catchments_secondary;
CREATE TABLE school_catchments_secondary (
    "USE_ID" INTEGER, 
    "CATCH_TYPE" VARCHAR(80), 
    "USE_DESC" VARCHAR(80), 
    "ADD_DATE" INTEGER, 
    "KINDERGART" VARCHAR(1),
    "YEAR1" VARCHAR(1),
    "YEAR2" VARCHAR(1),
    "YEAR3" VARCHAR(1),
    "YEAR4" VARCHAR(1),
    "YEAR5" VARCHAR(1),
    "YEAR6" VARCHAR(1),
    "YEAR7" VARCHAR(1),
    "YEAR8" VARCHAR(1),
    "YEAR9" VARCHAR(1),
    "YEAR10" VARCHAR(1),
    "YEAR11" VARCHAR(1),
    "YEAR12" VARCHAR(1),
    "PRIORITY" VARCHAR(80),
    "geom" GEOMETRY(MULTIPOLYGON,4326)
    
);"""
)


# BusinessStats
conn.execute("""
DROP TABLE IF EXISTS businessstats;
CREATE TABLE businessstats (
    "area_id" INTEGER,
    "area_name" VARCHAR(100),
    "number_of_businesses" INTEGER,
    "accommodation_and_food_services" INTEGER,
    "retail_trade" INTEGER,
    "agriculture_forestry_and_fishing" INTEGER,
    "health_care_and_social_assistance" INTEGER,
    "public_administration_and_safety" INTEGER,
    "transport_postal_and_warehousing" INTEGER
);"""
)

# Playgrounds
conn.execute("""
DROP TABLE IF EXISTS playgrounds;
CREATE TABLE playgrounds (
   "Name" VARCHAR(200),
   geom GEOMETRY(POINT, 4326)
);"""
)

# Lights
conn.execute("""
DROP TABLE IF EXISTS lights;
CREATE TABLE lights (
   "Location" VARCHAR(200),
   geom GEOMETRY(POINT, 4326)
);"""
)

In [ ]:
sa2_data.to_sql('sa2_data', conn, if_exists='append', index=False, dtype={'geom': Geometry('MULTIPOLYGON', srid)})
# query(conn,"select * from INFORMATION_SCHEMA.COLUMNS where TABLE_NAME='sa2_data'")
#query(conn, "select * from sa2_data")

In [ ]:
neighbourhoods.to_sql('neighbourhood', conn, if_exists='append', index=False)
#query(conn, "select * from neighbourhood")

In [ ]:
break_and_enter.to_sql("break_and_enter", conn, if_exists='append', index=False, dtype={'geom': Geometry('MULTIPOLYGON', srid)})
#query(conn, "select * from break_and_enter")

In [ ]:
school_catchments_future.to_sql("school_catchments_future", conn, if_exists='append', index=False, dtype={'geom': Geometry('MULTIPOLYGON', srid)})
#query(conn, "select * from school_catchments_future")

In [ ]:
school_catchments_primary.to_sql("school_catchments_primary", conn, if_exists='append', index=False, dtype={'geom': Geometry('MULTIPOLYGON', srid)})
#query(conn, "select * from school_catchments_primary")

In [ ]:
school_catchments_secondary.to_sql("school_catchments_secondary", conn, if_exists='append', index=False, dtype={'geom': Geometry('MULTIPOLYGON', srid)})
#query(conn, "select * from school_catchments_secondary")

In [ ]:
BusinessStats.to_sql('businessstats', conn, if_exists='append', index=False)
#query(conn, "select * from businessstats")

In [ ]:
Lights.to_sql('lights', conn, if_exists='append', index=False, dtype={'geom': Geometry('POINT', srid)})
#query(conn, "select * from lights")

In [ ]:
Playgrounds.to_sql('playgrounds', conn, if_exists='append', index=False, dtype={'geom': Geometry('POINT', srid)})
#query(conn, "select * from playgrounds")

In [ ]:
# query(conn,"select * from INFORMATION_SCHEMA.SCHEMATA")

In [ ]:
# select only the data relevant to greater sydney area

sql = """
select * from sa2_data where "GCC_NAME16" = 'Greater Sydney'
"""
query(conn, sql)

,SA2_MAIN16,SA2_5DIG16,SA2_NAME16,SA3_CODE16,SA3_NAME16,SA4_CODE16,SA4_NAME16,GCC_CODE16,GCC_NAME16,STE_CODE16,STE_NAME16,AREASQKM16,geom
0,102011042,11042,Woy Woy - Blackwall,10201,Gosford,102,Central Coast,1GSYD,Greater Sydney,1,New South Wales,17.4225,0106000020E6100000010000000103000000010000007E...
1,102011028,11028,Avoca Beach - Copacabana,10201,Gosford,102,Central Coast,1GSYD,Greater Sydney,1,New South Wales,6.4376,0106000020E6100000010000000103000000010000005E...
2,102011029,11029,Box Head - MacMasters Beach,10201,Gosford,102,Central Coast,1GSYD,Greater Sydney,1,New South Wales,32.0860,0106000020E61000000100000001030000000100000000...
3,102011030,11030,Calga - Kulnura,10201,Gosford,102,Central Coast,1GSYD,Greater Sydney,1,New South Wales,767.9512,0106000020E61000000200000001030000000100000085...
4,102011031,11031,Erina - Green Point,10201,Gosford,102,Central Coast,1GSYD,Greater Sydney,1,New South Wales,33.7934,0106000020E61000000100000001030000000100000041...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
307,128021537,11537,Royal National Park,12802,Sutherland - Menai - Heathcote,128,Sydney - Sutherland,1GSYD,Greater Sydney,1,New South Wales,139.3336,0106000020E61000000100000001030000000100000047...
308,128021538,11538,Sutherland - Kirrawee,12802,Sutherland - Menai - Heathcote,128,Sydney - Sutherland,1GSYD,Greater Sydney,1,New South Wales,7.7547,0106000020E61000000100000001030000000100000089...
309,128021607,11607,Engadine,12802,Sutherland - Menai - Heathcote,128,Sydney - Sutherland,1GSYD,Greater Sydney,1,New South Wales,8.9538,0106000020E6100000010000000103000000010000008E...
310,128021608,11608,Loftus - Yarrawarrah,12802,Sutherland - Menai - Heathcote,128,Sydney - Sutherland,1GSYD,Greater Sydney,1,New South Wales,3.8436,0106000020E610000001000000010300000001000000A1...


In [ ]:
# select the neighbourhoods with the highest businesses(from the neighbourhoods) in greater sydney area

sql = """
select S."SA2_MAIN16",S."SA2_NAME16", N."number_of_businesses"
from sa2_data S join neighbourhood N on (S."SA2_MAIN16" = N."area_id") 
where S."GCC_NAME16" = 'Greater Sydney'
ORDER BY N."number_of_businesses" DESC LIMIT 20
"""

query(conn, sql)

#  

,SA2_MAIN16,SA2_NAME16,number_of_businesses
0,123021439,Holsworthy Military Area,NaN
1,118011342,Centennial Park,NaN
2,124021456,Blue Mountains - South,NaN
3,117031337,Sydney - Haymarket - The Rocks,40285.0
4,121041417,North Sydney - Lavender Bay,6928.0
5,121011398,Chatswood (East) - Artarmon,5358.0
6,125041492,Parramatta - Rosehill,5067.0
7,118011343,Double Bay - Bellevue Hill,5016.0
8,115011291,Baulkham Hills (West) - Bella Vista,4674.0
9,117031336,Surry Hills,4302.0


In [ ]:
# School Catchments future

sql="""
select * from school_catchments_future S
"""
query(conn, sql)


,USE_ID,CATCH_TYPE,USE_DESC,ADD_DATE,KINDERGART,YEAR1,YEAR2,YEAR3,YEAR4,YEAR5,YEAR6,YEAR7,YEAR8,YEAR9,YEAR10,YEAR11,YEAR12,geom


In [ ]:
# school catchments primary

sql="""
select S."USE_ID", S."USE_DESC", C."SA2_NAME16"
from school_catchments_primary S join sa2_data C on
ST_Contains(C.geom, S.geom)


"""
query(conn, sql)

,USE_ID,USE_DESC,SA2_NAME16
0,4461,Guise PS,Macquarie Fields - Glenfield
1,4510,Claymore PS,Claymore - Eagle Vale - Raby
2,4509,Sarah Redfern PS,Minto - St Andrews
3,2852,Peats Ridge PS,Calga - Kulnura
4,4558,Eschol Park PS,Claymore - Eagle Vale - Raby
5,4125,Ermington WPS,Ermington - Rydalmere
6,4123,Rydalmere EPS,Ermington - Rydalmere
7,3871,Wattle Grove PS,Holsworthy - Wattle Grove
8,4651,Oran Park PS,Cobbitty - Leppington
9,4368,Wakehurst PS,Frenchs Forest - Belrose


In [ ]:
# school catchments secondary

sql="""
select S."USE_ID", S."USE_DESC", C."SA2_NAME16"
from school_catchments_secondary S join sa2_data C on
ST_Contains(C.geom, S.geom)
where C."GCC_NAME16" = 'Greater Sydney'

"""
query(conn, sql)


,USE_ID,USE_DESC,SA2_NAME16


In [ ]:
# businessstats area_id joined by sa2_data SA2_MAIN16

sql = """
select B."area_id", C."SA2_NAME16", B."number_of_businesses", B."accommodation_and_food_services"
from businessstats B 
join sa2_data C 
    on (B."area_id" = C."SA2_MAIN16")
where C."GCC_NAME16" = 'Greater Sydney'
order by B."number_of_businesses" desc limit 20
"""
query(conn, sql)


,area_id,SA2_NAME16,number_of_businesses,accommodation_and_food_services
0,117031337,Sydney - Haymarket - The Rocks,42126,1685
1,121041417,North Sydney - Lavender Bay,7124,222
2,121011398,Chatswood (East) - Artarmon,5659,284
3,125041492,Parramatta - Rosehill,5539,289
4,118011343,Double Bay - Bellevue Hill,5031,168
5,115011291,Baulkham Hills (West) - Bella Vista,4958,143
6,117031336,Surry Hills,4433,275
7,121041415,Mosman,4241,134
8,122031429,Freshwater - Brookvale,4226,117
9,117031334,Pyrmont - Ultimo,4158,231


In [ ]:
# number of retail services per thousand people
# businessstats - 

sql = """
select "area_id", "area_name", cast(replace(population, ',', '') as integer) as "population" from neighbourhood
"""

sql = """
select B."area_id", C."SA2_NAME16", B."retail_trade", N."population", ((CAST(B."retail_trade" as float))/ cast(replace(population, ',', '') as integer)*1000) AS "retail services per 1000 people"
from businessstats B 
join sa2_data C 
    on (B."area_id" = C."SA2_MAIN16")
join neighbourhood N 
    on (N."area_id" = C."SA2_MAIN16")
where C."GCC_NAME16" = 'Greater Sydney'
order by "retail services per 1000 people" desc limit 20

"""
query(conn, sql)


,area_id,SA2_NAME16,retail_trade,population,retail services per 1000 people
0,123021439,Holsworthy Military Area,0,None,NaN
1,127021521,Wetherill Park Industrial,86,46,1869.565217
2,117011324,Port Botany Industrial,9,7,1285.714286
3,117011320,Banksmeadow,26,21,1238.095238
4,125011475,Rookwood Cemetery,3,3,1000.000000
5,125031486,Smithfield Industrial,33,35,942.857143
6,116031318,Prospect Reservoir,21,40,525.000000
7,125031487,Yennora Industrial,9,23,391.304348
8,117031337,Sydney - Haymarket - The Rocks,1701,31010,54.853273
9,119011355,Chullora,63,1225,51.428571


Error encountered: 
(psycopg2.errors.GroupingError) column "groupedsuburbs.total_area" must appear in the GROUP BY clause or be used in an aggregate function
LINE 18: SELECT C."SA2_NAME16", GroupedSuburbs.total_area from sa2_da...
                                ^

[SQL: 

WITH GroupedSuburbs AS(
    select C."SA2_NAME16", SUM(ST_Area(T.geom)) AS "total_area"
    from  sa2_data C
    join break_and_enter T 
    on (ST_Contains(C.geom, T.geom))
    where C."GCC_NAME16" = 'Greater Sydney'
    group by C."SA2_NAME16"
),

GroupedSuburbsAvg AS(
    SELECT C."SA2_NAME16", 
    (GroupedSuburbs.total_area / ST_Area(C."geom")) AS "CPSA"
    FROM sa2_data C, GroupedSuburbs
)

SELECT C."SA2_NAME16", GroupedSuburbs.total_area from sa2_data C,GroupedSuburbs where C."GCC_NAME16" = 'Greater Sydney'
    group by C."SA2_NAME16";

]
(Background on this error at: https://sqlalche.me/e/14/f405)


""


In [ ]:
sql = """
select B."area_id", C."SA2_NAME16", B."health_care_and_social_assistance", N."population", ((CAST(B."health_care_and_social_assistance" as integer))/ cast(replace(population, ',', '') as integer)*1000) AS "health services per 1000 people"
from businessstats B 
join sa2_data C 
    on (B."area_id" = C."SA2_MAIN16")
join neighbourhood N 
    on (N."area_id" = C."SA2_MAIN16")
where C."GCC_NAME16" = 'Greater Sydney'
order by "health services per 1000 people" desc limit 20"""

query(conn, sql)

,area_id,SA2_NAME16,health_care_and_social_assistance,population,health services per 1000 people
0,123021439,Holsworthy Military Area,0,None,NaN
1,102011029,Box Head - MacMasters Beach,43,10986,0.0
2,102011043,Wyoming,48,11659,0.0
3,102011031,Erina - Green Point,179,14237,0.0
4,102011032,Gosford - Springfield,246,19385,0.0
5,102011033,Kariong,14,6540,0.0
6,102011034,Kincumber - Picketts Valley,49,7428,0.0
7,102011035,Narara,13,6853,0.0
8,102011036,Niagara Park - Lisarow,23,8173,0.0
9,102011037,Point Clare - Koolewong,30,6281,0.0


In [ ]:
conn.close()
db.dispose()